In [2]:
import os 
import re
import numpy as np 
import pandas as pd
from collections import OrderedDict

In [3]:
log_file = 'hdfs/HDFS_100k.log_structured.csv'
window='session'
window_size=0

assert window == 'session', "Only window=session is supported for HDFS dataset."
print("Loading", log_file)
struct_log = pd.read_csv(log_file, engine='c', na_filter=False, memory_map=True)
data_dict = OrderedDict()
hdfs_templates = OrderedDict()

Loading hdfs/HDFS_100k.log_structured.csv


In [4]:
for idx, row in struct_log.iterrows():
    if not row["EventId"] in hdfs_templates:
        hdfs_templates[row["EventId"]] = row["EventTemplate"]
hdfs_templates_df = pd.DataFrame(list(hdfs_templates.items()), columns=['EventId', 'EventTemplate'])
hdfs_templates_df.to_csv("hdfs/HDFS_templates.csv",index=None)

In [5]:
for idx, row in struct_log.iterrows():
    blkId_list = re.findall(r'(blk_-?\d+)', row['Content'])
    blkId_set = set(blkId_list)
    for blk_Id in blkId_set:
        if not blk_Id in data_dict:
            data_dict[blk_Id] = []
        data_dict[blk_Id].append(row['EventId'])


data_df = pd.DataFrame(list(data_dict.items()), columns=['BlockId', 'EventSequence'])


In [9]:
hdfs_templates_df

,EventId,EventTemplate
0,E5,Receiving block <*> src: /<*> dest: /<*>
1,E22,BLOCK* NameSystem.allocateBlock:<*>
2,E11,PacketResponder <*> for block <*> terminating
3,E9,Received block <*> of size <*> from /<*>
4,E26,BLOCK* NameSystem.addStoredBlock: blockMap upd...
5,E6,Received block <*> src: /<*> dest: /<*> of siz...
6,E16,<*>:Transmitted block <*> to /<*>
7,E18,<*> Starting thread to transfer block <*> to <*>
8,E25,BLOCK* ask <*> to replicate <*> to datanode(s)...
9,E3,<*> Served block <*> to /<*>
